此 ipynb 文件只用于内部演示，往 GitHub 提 PR 前会删除。

In [1]:
model_name = 'ced-tiny'  # ['ced-tiny', 'ced-mini', 'ced-small', 'ced-base']

In [2]:
# Convert checkpoint from original CED to 🤗 Transformers format
from transformers.models.ced.convert_ced_original_to_pytorch import convert_ced_checkpoint

model_saved_path = f'/home/zhangjunbo/sandbox/{model_name}'
convert_ced_checkpoint(model_name, model_saved_path)

Saving model ced-tiny to /home/zhangjunbo/sandbox/ced-tiny
Configuration saved in /home/zhangjunbo/sandbox/ced-tiny/config.json
Model weights saved in /home/zhangjunbo/sandbox/ced-tiny/pytorch_model.bin


In [3]:
import torch
import torchaudio

from transformers import CedConfig, CedForAudioClassification

In [4]:
config = CedConfig(model_name)

DEVICE = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model = CedForAudioClassification(config).from_pretrained(model_saved_path).to(DEVICE).eval()

loading configuration file /home/zhangjunbo/sandbox/ced-tiny/config.json
Model config CedConfig {
  "architectures": [
    "CedForAudioClassification"
  ],
  "attn_drop_rate": 0.0,
  "center": true,
  "depth": 12,
  "drop_path_rate": 0.0,
  "drop_rate": 0.0,
  "embed_dim": 192,
  "eval_avg": "mean",
  "f_max": 8000,
  "f_min": 0,
  "freq_patch_out": null,
  "hop_size": 160,
  "init_values": null,
  "mlp_ratio": 4,
  "n_fft": 512,
  "n_mels": 64,
  "num_heads": 3,
  "outputdim": 527,
  "pad_last": true,
  "patch_size": 16,
  "patch_stride": 16,
  "pooling": "mean",
  "qkv_bias": true,
  "target_length": 1012,
  "time_patch_out": null,
  "torch_dtype": "float32",
  "transformers_version": "4.34.0.dev0",
  "win_size": 512
}

loading weights file /home/zhangjunbo/sandbox/ced-tiny/pytorch_model.bin
All model checkpoint weights were used when initializing CedForAudioClassification.

All the weights of CedForAudioClassification were initialized from the model checkpoint at /home/zhangjunbo/sa

In [5]:
wavpath = '/home/zhangjunbo/src/CED/resources/JeD5V5aaaoI_931_932.wav'
wave, _ = torchaudio.load(wavpath)
wave = wave.to(DEVICE)

output = model(wave).squeeze(0)
output, output.shape

(tensor([1.6634e-03, 2.4463e-06, 2.9823e-07, 6.8086e-06, 5.7273e-09, 8.1335e-08,
         4.9931e-08, 1.0461e-07, 1.4798e-07, 1.0917e-07, 4.1153e-08, 1.4385e-08,
         3.5703e-08, 5.1580e-08, 4.3297e-08, 4.6092e-06, 5.3620e-08, 1.1018e-09,
         7.4081e-09, 1.2820e-09, 8.4438e-10, 8.2419e-09, 2.8360e-09, 2.6361e-09,
         4.8369e-08, 1.5871e-08, 6.5814e-07, 2.0008e-05, 9.4825e-06, 3.7283e-07,
         8.1454e-07, 5.5333e-06, 6.1662e-06, 1.2079e-06, 4.3939e-06, 1.3350e-06,
         2.3900e-07, 2.1458e-06, 3.2378e-07, 1.9709e-08, 3.5626e-04, 6.7121e-06,
         1.0083e-08, 1.2805e-06, 8.9587e-09, 2.2213e-06, 7.1053e-08, 9.1160e-09,
         1.5431e-09, 4.4430e-08, 9.0836e-07, 2.1307e-06, 1.3497e-05, 1.5031e-07,
         1.5489e-05, 6.8523e-07, 9.4678e-12, 4.4639e-06, 2.2572e-07, 1.0208e-07,
         5.8349e-06, 9.1606e-04, 8.8881e-01, 5.1715e-03, 1.9166e-05, 4.8840e-06,
         1.2344e-07, 9.2997e-05, 3.0151e-10, 1.0034e-06, 1.9051e-08, 1.2646e-08,
         4.3000e-03, 2.8627e

In [6]:
import pandas as pd

label_maps = pd.read_csv('http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv').set_index(
            'index')['display_name'].to_dict()

In [7]:
topk = 3

for k, (prob, label) in enumerate(zip(*output.topk(topk))):
    lab_idx = label.item()
    label_name = label_maps[lab_idx]
    print(f"Topk-{k+1} {label_name:<30} {prob:.4f}")

Topk-1 Finger snapping                0.8888
Topk-2 Slap, smack                    0.1020
Topk-3 Inside, small room             0.0365
